In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

betas = pd.read_csv("../data/processed/factor_betas.csv", index_col=0)
df = pd.read_csv("../data/processed/merged_dataset.csv", index_col=0, parse_dates=True)
df.head()

,AAPL,ABBV,ABT,ADBE,AMZN,AVGO,AXP,BA,BAC,BLK,...,V,WFC,XOM,Mkt-RF,SMB,HML,RMW,CMA,RF,Market
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-03,-0.099608,-0.032949,-0.047194,-0.039498,-0.025241,-0.088951,-0.019520,-0.039869,-0.016026,-0.029377,...,-0.036037,-0.007882,-0.015354,-0.0245,0.0048,0.0118,-0.0022,0.0091,0.0001,-0.0244
2019-01-04,0.042690,0.032217,0.028541,0.048632,0.050064,0.009828,0.045061,0.052042,0.041531,0.036616,...,0.043081,0.029633,0.036870,0.0355,0.0035,-0.0069,-0.0011,-0.0062,0.0001,0.0356
2019-01-07,-0.002226,0.014596,0.014975,0.013573,0.034353,0.020366,0.005428,0.003149,-0.000782,0.002782,...,0.018032,-0.006465,0.005200,0.0094,0.0087,-0.0075,-0.0074,-0.0045,0.0001,0.0095
2019-01-08,0.019063,0.004647,-0.012006,0.014918,0.016612,-0.008026,0.004889,0.037853,-0.001957,0.012726,...,0.005439,-0.002099,0.007271,0.0101,0.0044,-0.0063,0.0034,-0.0012,0.0001,0.0102
2019-01-09,0.016982,-0.032823,0.016691,0.011819,0.001714,0.043250,0.001825,0.009691,0.009800,0.005730,...,0.011769,0.005469,0.005275,0.0052,0.0050,0.0011,0.0009,-0.0020,0.0001,0.0053


In [2]:
#the premium price paid to take the risk / Options
factor_cols = ["Market", "SMB", "HML", "RMW", "CMA"]

premium = df[factor_cols].mean()
premium

Market    0.000706
SMB      -0.000091
HML      -0.000083
RMW       0.000246
CMA      -0.000013
dtype: float64

In [3]:
annual_premiums = (1 + premium)**252 - 1
annual_premiums

Market    0.194555
SMB      -0.022571
HML      -0.020692
RMW       0.064032
CMA      -0.003235
dtype: float64

In [4]:
expected_returns = pd.Series(dtype=float)

for stock in betas.index:
    alpha = betas.loc[stock, "const"] 
    ii = betas.loc[stock, factor_cols]
    
    exp_return = alpha + np.dot(ii, annual_premiums)
    
    expected_returns.loc[stock] = exp_return

expected_returns.head()

AAPL    0.290227
ABBV    0.126451
ABT     0.158700
ADBE    0.253379
AMZN    0.208973
dtype: float64

In [5]:
expected_returns = expected_returns.sort_values(ascending=False)
expected_returns.to_csv("../data/results/expected_returns.csv")

expected_returns.head(20)

NVDA      0.364163
AVGO      0.296053
AAPL      0.290227
META      0.265266
MSFT      0.264660
ADBE      0.253379
GOOGL     0.238380
MA        0.235503
BA        0.234817
TSLA      0.227287
HD        0.220511
BLK       0.217028
AXP       0.214303
SBUX      0.210057
AMZN      0.208973
NKE       0.208093
V         0.205499
MS        0.203356
GE        0.202399
Mkt-RF    0.194496
dtype: float64

In [6]:
factor_cols = ["Market", "SMB", "HML", "RMW", "CMA"]

beta_nvda = betas.loc["NVDA", factor_cols]
alpha_nvda = betas.loc["NVDA", "const"]

contrib_nvda = beta_nvda * annual_premiums

nvda_breakdown = contrib_nvda.copy()
nvda_breakdown["Alpha"] = alpha_nvda
nvda_breakdown["Total_without_alpha"] = contrib_nvda.sum()
nvda_breakdown

Market                 0.332083
SMB                    0.004311
HML                    0.016396
RMW                    0.008266
CMA                    0.001458
Alpha                  0.001648
Total_without_alpha    0.362514
dtype: float64

In [ ]:
expected_nvda = alpha_nvda + contrib_nvda.sum()
expected_nvda, expected_returns["NVDA"]

In [ ]:
import matplotlib.pyplot as plt

plot_series = contrib_nvda.copy()
plot_series["Alpha"] = alpha_nvda

plt.figure(figsize=(8,5))
plt.bar(plot_series.index, plot_series.values)
plt.axhline(0, linewidth=1)
plt.title("NVDA — Expected Return Breakdown by Factor")
plt.ylabel("Contribution to Annual Expected Return")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
